In [ ]:
# from urllib.request import Request, urlopen
from urllib.parse import urlparse
import urllib.robotparser
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as fOptions
from selenium.webdriver.common.by import By
import re
import random
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import psycopg2
import psycopg2.errorcodes as errorcodes
import concurrent.futures
import threading
# new packages
import mmh3
from ordered_set import OrderedSet
import pickle
import warnings
from IPython.display import clear_output
warnings.filterwarnings('ignore')
# http = urllib.PoolManager(num_pools=50)

In [ ]:
TIMEOUT = 5
num_sites = 0
numHashes = 25
maxShingleID = 2**64-1
nextPrime = 1.8446844e+19

with open("coeffA.pickle", "rb") as f:
    coeffA = pickle.load(f)
    
with open("coeffB.pickle", "rb") as f:
    coeffB = pickle.load(f)
    
seed_sites_ids = {}
with open("seed_sites_ids_latest.pickle", "rb") as f:
    seed_sites_ids = pickle.load(f)
    
with open("duplicate_urls_latest.pickle", "rb") as f:
    duplicate_urls = pickle.load(f)

lock = threading.Lock()

seed_sites = set()
with open("seed_sites_latest.pickle", "rb") as f:
    seed_sites.update(pickle.load(f))

frontier = OrderedSet()

with open("signatures_latest.pickle", "rb") as f:
    signatures = pickle.load(f)


signatureForLink = {}
with open("signatureForLink_latest.pickle", "rb") as f:
    signatureForLink = pickle.load(f)
    
robot_parser = {}
with open("robot_parser_latest.pickle", "rb") as f:
    robot_parser = pickle.load(f)
    
duplicate_binaries = set()

with open("duplicate_binaries_latest.pickle", "rb") as f:
    duplicate_binaries.update(pickle.load(f))

In [ ]:
find_page_url_query= '''SELECT id, url FROM crawldb.page WHERE page_type_code = %s'''
get_page_content= '''SELECT id, url, html_content FROM crawldb.page WHERE page_type_code = %s'''
site_insert_query = '''INSERT INTO crawldb.site(domain, robots_content, sitemap_content) VALUES (%s,%s,%s) RETURNING id'''
page_insert_query = '''INSERT INTO crawldb.page(site_id, page_type_code, url, html_content, http_status_code, accessed_time) VALUES (%s,%s,%s,%s,%s,%s) RETURNING id'''
page_data_insert_query = ''' INSERT INTO crawldb.page_data(page_id, data_type_code, data) VALUES (%s,%s,%s)'''
image_insert_query = ''' INSERT INTO crawldb.image(page_id, filename, content_type, accessed_time,data) VALUES (%s,%s,%s,%s,%s) RETURNING id'''
link_insert_query = ''' INSERT INTO crawldb.link(from_page, to_page) VALUES (%s,%s)'''

find_page_id_query = '''SELECT id FROM crawldb.page WHERE url = %s'''
update_page_query = ''' UPDATE crawldb.page SET page_type_code=%s, html_content=%s, http_status_code=%s,accessed_time=%s WHERE id=%s'''
update_page_content = ''' UPDATE crawldb.page SET page_type_code=%s, html_content=%s WHERE id=%s'''
get_frontier_query = """SELECT id, url FROM crawldb.page WHERE page_type_code='FRONTIER' and not (url like '%bold_mode%' or url like '%caps_mode%' or url like '%view_mode%' or url like '%login%' or url like '%jpg%' or url like '%auth%' or url like '%download%' or url like '%file%')"""

In [ ]:
with open ('frontier_lodi.pickle', 'rb') as fp:
    skip_frontier = pickle.load(fp)
# TIMEOUT = 5

In [ ]:
conn = psycopg2.connect(host="localhost", user="postgres", password="postgres")
conn.autocommit = True
cursor = conn.cursor()
cursor.execute(get_frontier_query)
p = cursor.fetchall()
frontier = OrderedSet()
frontier.update(p)
print(len(frontier))
cursor.close()
conn.close()

In [ ]:
# For each of the 'numHashes' hash functions, generate a different coefficient 'a' and 'b'.   
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

def generate_shingles(tokens):
    shinglesInHTML = set()
    for index in range(0, len(tokens) - 2):
        shingle = "{} {} {}".format(tokens[index], tokens[index + 1], tokens[index + 2])
        hashed_shingle = mmh3.hash64(shingle)[0] & 0xffffffffffffffff
        shinglesInHTML.add(hashed_shingle)
#     print(shinglesInHTML)
    return shinglesInHTML

def minhash(shingleIDSet):

    # The resulting minhash signature for this document. 
    signature = []

    # For each of the random hash functions...
    for i in range(0, numHashes):

        # For each of the shingles actually in the document, calculate its hash code
        # using hash function 'i'. 

        # Track the lowest hash ID seen. Initialize 'minHashCode' to be greater than
        # the maximum possible value output by the hash.
        minHashCode = nextPrime + 1

        # For each shingle in the document...
        for shingleID in shingleIDSet:
            # Evaluate the hash function.
#             print(shingleID.bit_length())
            hashCode = (coeffA[i] * shingleID + coeffB[i]) % nextPrime 

            # Track the lowest hash code seen.
            if hashCode < minHashCode:
                minHashCode = hashCode

        # Add the smallest hash code value as component number 'i' of the signature.
        signature.append(minHashCode)

    # Store the MinHash signature for this document.
    return signature

In [ ]:
def compare_signatures(signature):
    t0 = time.time()
    duplicate_signature = None
    count = 0
    # For each of the test documents...
    for signature2 in signatures:
        count = 0
        for k in range(0, numHashes):
            count += (signature[k] == signature2[k])
        if count/numHashes > 0.8:
            break
    print(f"HASH: {count/numHashes}")
    # Calculate the elapsed time (in seconds)
    elapsed = (time.time() - t0)

    print ("Comparing MinHash signatures took %.2fsec" % elapsed)
    return signature2 if count/numHashes > 0.8 else None

  

In [ ]:
binary_file_matcher = re.compile("(\.doc|\.docx|\.xls|\.xlsx|\.pdf|\.ppt|\.pptx|\.png|\.jpg|\.jpeg|\.gif)$")
tmp_frontier = OrderedSet()
for vv in frontier:
    if vv[1] not in skip_frontier and not binary_file_matcher.search(vv[1]) and not ("mailto:" in vv[1] or "zip" in vv[1] or ".xls" in vv[1] or "pdf" in vv[1]\
                or ".csv" in vv[1] or ".odt" in vv[1] or ".ods" in vv[1] or "jpg" in vv[1]\
                or "bold_mode" in vv[1] or "view_mode" in vv[1] or "file" in vv[1]\
                or "dokument" in vv[1] or "caps_mode" in vv[1] or "download" in vv[1]\
                or "login" in vv[1] or "auth" in vv[1] or "ppt" in vv[1]\
                or ".crt" in vv[1] or ".pem" in vv[1] or ".crl" in vv[1] ):
        tmp_frontier.add(vv)
frontier = OrderedSet() | tmp_frontier
print(len(frontier))



In [ ]:
signatures = []
signatureForLink = {}
# conn = psycopg2.connect(host="localhost", user="postgres", password="postgres")
# conn.autocommit = True
# original_parent_id = None
for content in page_content:
    tokens = preprocess(content[2])
    shingles_in_html = generate_shingles(tokens)
    signature = minhash(shingles_in_html)
    p = compare_signatures(signature)

    signatures.append(signature)
    signature_copy = signature.copy()
    signature_copy.sort()
    mapped_sorted_os = map(str, signature_copy)
    string_sorted_os = ",".join(mapped_sorted_os)
    signatureForLink[string_sorted_os] = content[1] 
# conn.close()

In [ ]:
def driver_info(name):
    if name == "Lodi":
#         WEB_DRIVER_LOCATION = "D:/FRI/Summer Semester 2020/Web information extraction and retrieval/Assignment 1/geckodriver-v0.26.0-win64/geckodriver.exe"
#         options = fOptions()
#         driver = webdriver.Firefox(executable_path=WEB_DRIVER_LOCATION,options=options)
        WEB_DRIVER_LOCATION = "D:/FRI/Summer Semester 2020/Web information extraction and retrieval/Assignment 1/geckodriver-v0.26.0-win64/geckodriver.exe"
        options = fOptions()
        # If you comment the following line, a browser will show ...
        options.add_argument("--headless")
        #Adding a specific user agent
        options.add_argument("user-agent=fri-ieps-26")
        driver = webdriver.Firefox(executable_path=WEB_DRIVER_LOCATION,options=options)        
        
        
    elif name == "Viktor":
        WEB_DRIVER_LOCATION = "/home/viktor/Documents/FRI/WIER/Assignments/geckodriver"
        options = fOptions()
        # If you comment the following line, a browser will show ...
        options.add_argument("--headless")
        #Adding a specific user agent
        options.add_argument("user-agent=fri-ieps-26")
        driver = webdriver.Firefox(executable_path=WEB_DRIVER_LOCATION,
                               options=options)
          
    return driver

In [ ]:
def get_response_code(url):
    if "mailto:" in url:
        print("skipping: e-mail")
        return 503, None, []
    try:
        response = requests.head(url, verify=False, allow_redirects=True, timeout=1) 
        return response.status_code, response.url, response.history
    except Exception:
        print("Timeout exception for url: ", url)
        return 503, url, []

In [ ]:
def extract_content(page_id_parent, parent_url):
    parent_url = parent_url.replace("www.", "")
    parsed_url = urlparse(parent_url)
    rp = None
    try:
        rp = robot_parser[parsed_url.netloc]
    except Exception:
        pass
    if rp is None or not rp.can_fetch("fri-ieps-26",parent_url):
        return
    conn = psycopg2.connect(host="localhost", user="postgres", password="postgres")
    conn.autocommit = True
    driver = driver_info("Viktor")
    accessed_time_parent_url = datetime.now()
    print(f"Retrieving web page URL '{parent_url}'")
    driver.get(parent_url)
    
     # Timeout needed for Web page to render (read more about it)
    time.sleep(TIMEOUT)
    images = driver.find_elements_by_tag_name("img")
    for image in images:
        src = image.get_attribute("src")
        img_id = None
        page_id = None
        add_link_flag = False
        if src:
            gov_si_matcher = re.match("(http|https)://(.*)\.?gov\.si(.*)",src)
            if gov_si_matcher:
                if src not in duplicate_binaries:
                    image_sources.add(src)
                    duplicate_binaries.add(src)
                    accessed_time = datetime.now()
                    http_status_code, http_redirect_url, res_history = get_response_code(url)
                    time.sleep(TIMEOUT)

                     # ADD CURRENT IMAGE TO PAGE DB
                    try:
                        with lock:
                            cursor = conn.cursor()
                            cursor.execute(page_insert_query, (site_id, "BINARY", src, html_content_binary, http_status_code, accessed_time))
                            page_id = cursor.fetchone()[0]
                            conn.commit()
                            add_link_flag = True
                            cursor.close()

                    except (Exception, psycopg2.DatabaseError) as error :
                        if error.pgcode == errorcodes.UNIQUE_VIOLATION or error.pgcode == "23505" or "duplicate" in error:
                            pass
                        else:
                            print ("Error while writing binary page", src, " to DB: ", error)
                
                if page_id is not None:

                    # ADD CURRENT IMAGE TO IMAGE DB
                    file_name = src.rsplit("/", 1)[-1]
                    filename= file_name.split('.')[0]
                    content_type = file_name.split('.')[-1].lower()
                    if content_type in img_types_dict:
                        content_type = img_types_dict[content_type]
                    else:
                        content_type = None
                    try:
                        with lock:
                            cursor = conn.cursor()
                            cursor.execute(image_insert_query, (page_id, filename, content_type, accessed_time, html_content_binary))
                            img_id = cursor.fetchone()[0]
                            conn.commit()
                            cursor.close()
                            
                    except (Exception, psycopg2.DatabaseError) as error :
                        print ("Error while writing image ", src, " to DB: ", error)

                    if add_link_flag and page_id_parent is not None:
                    # ADD LINKS TO DB
                        try:
                            with lock:
                                cursor = conn.cursor()
                                cursor.execute(link_insert_query, (page_id_parent, page_id))
                                conn.commit()
                                cursor.close()
                        except (Exception, psycopg2.DatabaseError) as error :
                            print ("Error while writing link from : ", parent_url, " to ",src," :", error)
    html = driver.page_source
    
    # extract the text from the raw html (everything below)
    soup = BeautifulSoup(html, "html.parser")
    [
        s.extract()
        for s in soup(['style', 'script', '[document]', 'head', 'title'])
    ]
    text = soup.getText()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    content = '\n'.join(chunk for chunk in chunks if chunk)
    tokens = preprocess(content)
    shingles_in_html = generate_shingles(tokens)
    signature = minhash(shingles_in_html)
    original_signature = compare_signatures(signature)
    code = "HTML"
    original_parent_id = None
    if original_signature is not None:
        code = "DUPLICATE"
        content = None
        original_signature.sort()
        mapped_sorted_os = map(str, original_signature)
        string_sorted_os = ",".join(mapped_sorted_os)
        original_url = signatureForLink[string_sorted_os]
        try:
            with lock:
                cursor = conn.cursor()
                cursor.execute(find_page_id_query,(original_url,))
                original_parent_id = cursor.fetchone()[0]
                cursor.close()
        except (Exception, psycopg2.DatabaseError) as error :
            print ("Error while trying to fetch page id of: ", original_url, ": ", error)
        try:
            with lock:
                cursor = conn.cursor()
                cursor.execute(link_insert_query, (original_parent_id, page_id_parent))
                conn.commit()
                cursor.close()
        except (Exception, psycopg2.DatabaseError) as error :
            print ("Error while writing duplicate link from : ", original_parent_id, " to ",page_id_parent," :", error)
    else:
        signatures.append(signature)
        signature_copy = signature.copy()
        signature_copy.sort()
        mapped_sorted_os = map(str, signature_copy)
        string_sorted_os = ",".join(mapped_sorted_os)
        signatureForLink[string_sorted_os] = parent_url 
    
    if page_id_parent is not None:
        # The page is already in the DB. We need to update the information.
        print("UPDATING PARENT: ", parent_url)
        http_status_code, http_redirect_url, res_history = get_response_code(parent_url)
        time.sleep(TIMEOUT)
        try:
            with lock:
                cursor = conn.cursor()
                cursor.execute(update_page_query,(code,content,http_status_code,accessed_time_parent_url,page_id_parent))
                cursor.close()
                conn.commit()
                
        except (Exception, psycopg2.DatabaseError) as error :
            print ("ERROR WHILE UPDATING PAGE ", parent_url," : ", error)
    driver.close()
    conn.close()

In [ ]:
i = 0
while i <= len(frontier):
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        print(f"\n ... executing workers ...\n")
        for _ in range(20):
            executor.submit(extract_content, frontier[i][0], frontier[i][1])
            i += 1
        print(f"i: {i}, frontier: {len(frontier)}")
    clear_output(wait=True)
        